# * Ex-D Data

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
# v_start_date = 20250201
print(f'\nParameter input...\n')
# print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        CASE WHEN REGEXP_LIKE(METRIC_CD, '^DB1R000900|^DB1R001000|^DB1S000101|^DB1S000109|^DB2R000500|^DB2S000100') 
            OR METRIC_CD IN (
                'DB0R00010001' --Total Inflow M1 : DTAC'
                , 'DB1S000100' --Prepay New sub
                , 'DB2S010100' --Postpaid activation sub B2C
                , 'DB2S020100' --Postpaid activation sub B2B
                --, 'DB1S000102' --Prepaid Activation Subs : Thai Mass : DTAC
                --, 'DB1S000103' --Prepaid Activation Subs : AEC/Migrants : DTAC
                --, 'DB1S000104' --Prepaid Activation Subs : Tourists (IN + OUT) : DTAC
                ) THEN 'Obsolete'
            ELSE 'Active'
            END KPI_FLAG
        , PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
        , CASE WHEN REGEXP_LIKE(METRIC_CD, 'A[A-K]$') THEN SUBSTR(METRIC_CD, -2) ELSE 'ALL' END CHANNEL_CD
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) START_DAY, MAX(TM_KEY_DAY) END_DAY
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    GROUP BY PAR_KEY, BASENAME, METRIC_CD, METRIC_NAME
    --ORDER BY 1, PAR_KEY, BASENAME, METRIC_CD 
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...


TDMDBPR : Connected

   -> Execute query... 2025-03-21, 10:27:04

   -> Summany DataFrame : 131 rows, 20 columns

TDMDBPR : Disconnected


In [3]:
''' Revenue '''

# curr_mth = chk_src_df['TM_KEY_MTH'].max()
src_d_kpi_flag = 'Active'
rev_metric_list = [
    'DB1R000100' #Prepay Revenue
    , 'DB2R000100' #Postpaid Revenue
    , 'DB2R010100' #Postpaid Revenue B2C
    , 'DB2R020100' #Postpaid Revenue B2B
    ]

src_d_rev_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
# src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_d_rev_overall_df = src_d_rev_overall_df.loc[src_d_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_d_rev_overall_df = src_d_rev_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_rev_overall_df = src_d_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_rev_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_rev_overall_df[col] = src_d_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_rev_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250318,DB1R000100,Prepay Revenue,2025-03-18,20240528,20250318,"5,658,300,891","10,197,389,491","10,189,975,068","10,189,975,068","5,658,300,599","94,134,697",5,1100,107580
1,20250318,DB2R000100,Postpaid Revenue,2025-03-18,20240528,20250318,"8,882,870,946","15,961,636,611","15,948,342,805","15,948,342,805","8,882,870,531","250,708,835",5,1100,107002
2,20250318,DB2R010100,Postpaid Revenue B2C,2025-03-18,20240528,20250318,"8,116,815,597","14,608,688,136","14,597,183,461","14,597,183,461","8,116,815,182","234,758,054",5,1100,106945
3,20250318,DB2R020100,Postpaid Revenue B2B,2025-03-18,20240528,20250318,"766,055,349","1,352,948,474","1,351,159,344","1,351,159,344","766,055,349","15,950,781",5,1100,69154


In [4]:
''' Subscription '''

src_d_kpi_flag = 'Active'
sub_metric_list = [
    'DB1S000500' #Prepay Usage Subs
    , 'DB1S000600' #Prepay Revenue Subs
    , 'DB2S000500' #Postpaid Active Subs
    , 'DB2S010500' #Postpaid Active Subs B2C
    , 'DB2S020400' #Postpaid Active Sub B2B
    ]

src_d_sub_overall_df = chk_src_df.loc[chk_src_df['KPI_FLAG']==src_d_kpi_flag].copy()
src_d_sub_overall_df = src_d_sub_overall_df.loc[src_d_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_d_sub_overall_df = src_d_sub_overall_df.groupby(['PAR_KEY', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'H_080':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_d_sub_overall_df = src_d_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_d_sub_overall_df.iloc[:, 6:12].columns.tolist()
for col in mod_col_list:
    src_d_sub_overall_df[col] = src_d_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_d_sub_overall_df

,PAR_KEY,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,P,G,H,HH,CCAA,H_080,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250318,DB1S000500,Prepay Usage Subs,2025-03-18,20240528,20250318,"1,153,671,995","2,068,674,395","2,067,227,826","2,067,227,826","1,153,671,839","19,969,546",5,1100,107564
1,20250318,DB1S000600,Prepay Revenue Subs,2025-03-18,20240528,20250318,"609,516,269","1,097,064,970","1,096,214,281","1,096,214,281","609,516,212","10,650,133",5,1100,107564
2,20250318,DB2S000500,Postpaid Active Subs,2025-03-18,20240528,20250318,"565,171,155","1,015,347,315","1,014,485,584","1,014,485,584","565,171,012","15,089,511",5,1100,107389
3,20250318,DB2S010500,Postpaid Active Subs B2C,2025-03-18,20240528,20250318,"499,130,645","896,216,693","895,539,758","895,539,758","499,130,502","13,607,542",5,1100,107389
4,20250318,DB2S020400,Postpaid Active Sub B2B,2025-03-18,20240528,20250318,"66,040,510","119,130,622","118,945,826","118,945,826","66,040,510","1,481,969",5,1100,106787


## Actual : Transaction
->  STG_KPI_NEWCO_DIAMOND_ACTUAL_INC

In [5]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_CD = 'P' THEN METRIC_VALUE END) AS P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN METRIC_VALUE END) AS G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN METRIC_VALUE END) AS H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN METRIC_VALUE END) AS HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN METRIC_VALUE END) AS CCAA
        , SUM(CASE WHEN AREA_CD = '080' THEN METRIC_VALUE END) AS H_080
        , SUM(CASE WHEN AREA_CD = '016' THEN METRIC_VALUE END) AS H_016
        , SUM(CASE WHEN AREA_CD = '040' THEN METRIC_VALUE END) AS H_040
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM GEOSPCAPPO.STG_KPI_NEWCO_DIAMOND_ACTUAL_INC
    WHERE METRIC_CD IN (
        'DB1R000100' --Prepay Revenue
        , 'DB2R000100' --Postpaid Revenue
        , 'DB2R010100' --Postpaid Revenue B2C
        , 'DB2R020100' --Postpaid Revenue B2B
        , 'DB1S000500' --Prepay Usage Subs
        , 'DB1S000600' --Prepay Revenue Subs
        , 'DB2S000500' --Postpaid Active Subs
        , 'DB2S010500' --Postpaid Active Subs B2C
        , 'DB2S020400' --Postpaid Active Sub B2B
        )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY PAR_KEY, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summary DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-03-21, 10:27:31

   -> Daily DataFrame : 864 rows, 16 columns

TDMDBPR : Disconnected


In [6]:
''' DB1R000100 : Prepay Revenue '''

v_start_day = 20240101
v_metric_cd = 'DB1R000100'
print(f'\nDB1R000100 : Prepay Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB1R000100_day__df = chk_src_df.copy()
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB1R000100_day__df = DB1R000100_day__df.loc[DB1R000100_day__df['METRIC_CD']==v_metric_cd]
DB1R000100_day__df = DB1R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB1R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB1R000100_day__df[col] = DB1R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB1R000100_day__df


DB1R000100 : Prepay Revenue

   -> v_metric_cd: DB1R000100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250318,DB1R000100,Prepay Revenue,20240528,2024-06-06,"57,081,475","57,081,475","57,081,459","57,081,459","57,081,459",0,"303,238","890,374",5,1099,1111
1,20250318,DB1R000100,Prepay Revenue,20240529,2024-06-07,"54,750,640","54,750,640","54,750,601","54,750,601","54,750,601",0,"320,514","861,052",5,1099,1111
2,20250318,DB1R000100,Prepay Revenue,20240530,2024-06-08,"59,115,506","59,115,506","59,115,490","59,115,490","59,115,490",0,"312,960","934,515",5,1099,1111
3,20250318,DB1R000100,Prepay Revenue,20240531,2024-06-09,"70,858,400","70,858,400","70,858,378","70,858,378","70,858,378",0,"382,069","1,193,650",5,1099,1111
4,20250318,DB1R000100,Prepay Revenue,20240601,2024-06-10,"70,350,955","70,350,955","70,350,942","70,350,942","70,350,942",0,"385,655","1,166,659",5,1099,1111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,20250318,DB1R000100,Prepay Revenue,20250314,2025-03-18,"52,479,576","104,959,152","104,881,520","104,881,520","52,479,576","1,058,544",0,0,5,1098,1123
92,20250318,DB1R000100,Prepay Revenue,20250315,2025-03-18,"53,847,434","107,694,868","107,615,279","107,615,279","53,847,434","1,145,118",0,0,5,1098,1123
93,20250318,DB1R000100,Prepay Revenue,20250316,2025-03-18,"54,844,302","109,688,604","109,592,324","109,592,324","54,844,302","1,202,608",0,0,5,1098,1123
94,20250318,DB1R000100,Prepay Revenue,20250317,2025-03-18,"54,534,269","109,068,537","108,981,911","108,981,911","54,534,269","1,144,034",0,0,5,1098,1123


In [7]:
''' DB2R000100 : Postpaid Revenue '''

v_start_day = 20240101
v_metric_cd = 'DB2R000100'
print(f'\nDB2R000100 : Postpaid Revenue\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R000100_day__df = chk_src_df.copy()
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R000100_day__df = DB2R000100_day__df.loc[DB2R000100_day__df['METRIC_CD']==v_metric_cd]
DB2R000100_day__df = DB2R000100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R000100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R000100_day__df[col] = DB2R000100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R000100_day__df


DB2R000100 : Postpaid Revenue

   -> v_metric_cd: DB2R000100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250318,DB2R000100,Postpaid Revenue,20240528,2024-06-06,"93,542,212","93,542,212","93,542,212","93,542,212","93,542,212",0,"1,056,671","2,289,788",5,1099,1108
1,20250318,DB2R000100,Postpaid Revenue,20240529,2024-06-07,"281,666,951","281,666,951","281,666,951","281,666,951","281,666,951",0,"3,264,840","6,703,783",5,1099,1108
2,20250318,DB2R000100,Postpaid Revenue,20240530,2024-06-08,"6,554,985","6,554,985","6,554,985","6,554,985","6,554,985",0,"72,122","128,142",5,1099,1102
3,20250318,DB2R000100,Postpaid Revenue,20240531,2024-06-09,"6,595,697","6,595,697","6,595,697","6,595,697","6,595,697",0,"87,719","144,124",5,1099,1103
4,20250318,DB2R000100,Postpaid Revenue,20240601,2024-06-10,"8,733,426","8,733,426","8,733,426","8,733,426","8,733,426",0,"90,050","188,734",5,1099,1103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,20250318,DB2R000100,Postpaid Revenue,20250314,2025-03-18,"80,448,548","160,897,097","160,743,591","160,743,591","80,448,548","2,801,055",0,0,5,1098,1117
92,20250318,DB2R000100,Postpaid Revenue,20250315,2025-03-18,"79,187,989","158,375,978","158,223,708","158,223,708","79,187,989","2,879,808",0,0,5,1098,1119
93,20250318,DB2R000100,Postpaid Revenue,20250316,2025-03-18,"81,360,499","162,720,999","162,585,882","162,585,882","81,360,499","2,925,632",0,0,5,1098,1119
94,20250318,DB2R000100,Postpaid Revenue,20250317,2025-03-18,"79,990,469","159,980,939","159,835,976","159,835,976","79,990,469","2,742,257",0,0,5,1097,1115


In [8]:
''' DB2R010100 : Postpaid Revenue B2C '''

v_start_day = 20240101
v_metric_cd = 'DB2R010100'
print(f'\nDB2R010100 : Postpaid Revenue B2C\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R010100_day__df = chk_src_df.copy()
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R010100_day__df = DB2R010100_day__df.loc[DB2R010100_day__df['METRIC_CD']==v_metric_cd]
DB2R010100_day__df = DB2R010100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R010100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R010100_day__df[col] = DB2R010100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R010100_day__df


DB2R010100 : Postpaid Revenue B2C

   -> v_metric_cd: DB2R010100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250318,DB2R010100,Postpaid Revenue B2C,20240528,2024-06-06,"90,417,597","90,417,597","90,417,597","90,417,597","90,417,597",0,"1,054,690","2,284,168",5,1098,1107
1,20250318,DB2R010100,Postpaid Revenue B2C,20240529,2024-06-07,"280,520,688","280,520,688","280,520,688","280,520,688","280,520,688",0,"3,262,211","6,695,846",5,1099,1108
2,20250318,DB2R010100,Postpaid Revenue B2C,20240530,2024-06-08,"4,883,710","4,883,710","4,883,710","4,883,710","4,883,710",0,"70,826","122,243",5,1099,1101
3,20250318,DB2R010100,Postpaid Revenue B2C,20240531,2024-06-09,"5,308,594","5,308,594","5,308,594","5,308,594","5,308,594",0,"86,421","134,865",5,1099,1101
4,20250318,DB2R010100,Postpaid Revenue B2C,20240601,2024-06-10,"5,843,787","5,843,787","5,843,787","5,843,787","5,843,787",0,"84,771","173,885",5,1099,1102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,20250318,DB2R010100,Postpaid Revenue B2C,20250314,2025-03-18,"77,373,842","154,747,683","154,594,302","154,594,302","77,373,842","2,794,442",0,0,5,1098,1117
92,20250318,DB2R010100,Postpaid Revenue B2C,20250315,2025-03-18,"78,081,457","156,162,914","156,010,997","156,010,997","78,081,457","2,870,301",0,0,5,1098,1119
93,20250318,DB2R010100,Postpaid Revenue B2C,20250316,2025-03-18,"78,580,635","157,161,271","157,026,224","157,026,224","78,580,635","2,920,571",0,0,5,1098,1119
94,20250318,DB2R010100,Postpaid Revenue B2C,20250317,2025-03-18,"77,853,959","155,707,919","155,562,853","155,562,853","77,853,959","2,730,055",0,0,5,1097,1114


In [9]:
''' DB2R020100 : Postpaid Revenue B2B '''

v_start_day = 20240101
v_metric_cd = 'DB2R020100'
print(f'\nDB2R020100 : Postpaid Revenue B2B\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_start_day: {v_start_day}')

DB2R020100_day__df = chk_src_df.copy()
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['TM_KEY_DAY']>=v_start_day]
DB2R020100_day__df = DB2R020100_day__df.loc[DB2R020100_day__df['METRIC_CD']==v_metric_cd]
DB2R020100_day__df = DB2R020100_day__df.fillna(0).sort_values(by=['METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = DB2R020100_day__df.iloc[:, 5:13].columns.tolist()
for col in mod_col_list:
    DB2R020100_day__df[col] = DB2R020100_day__df[col].apply(lambda x: format(x, ',.0f'))

DB2R020100_day__df


DB2R020100 : Postpaid Revenue B2B

   -> v_metric_cd: DB2R020100
   -> v_start_day: 20240101


,PAR_KEY,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,P,G,H,HH,CCAA,H_080,H_016,H_040,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,20250318,DB2R020100,Postpaid Revenue B2B,20240528,2024-06-06,"3,124,614","3,124,614","3,124,614","3,124,614","3,124,614",0,"1,981","5,620",5,726,729
1,20250318,DB2R020100,Postpaid Revenue B2B,20240529,2024-06-07,"1,146,263","1,146,263","1,146,263","1,146,263","1,146,263",0,"2,629","7,936",5,734,737
2,20250318,DB2R020100,Postpaid Revenue B2B,20240530,2024-06-08,"1,671,275","1,671,275","1,671,275","1,671,275","1,671,275",0,"1,296","5,899",5,740,743
3,20250318,DB2R020100,Postpaid Revenue B2B,20240531,2024-06-09,"1,287,104","1,287,104","1,287,104","1,287,104","1,287,104",0,"1,298","9,258",5,780,783
4,20250318,DB2R020100,Postpaid Revenue B2B,20240601,2024-06-10,"2,889,639","2,889,639","2,889,639","2,889,639","2,889,639",0,"5,279","14,848",5,731,734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,20250318,DB2R020100,Postpaid Revenue B2B,20250314,2025-03-18,"3,074,707","6,149,413","6,149,290","6,149,290","3,074,707","6,613",0,0,5,739,754
92,20250318,DB2R020100,Postpaid Revenue B2B,20250315,2025-03-18,"1,106,532","2,213,064","2,212,711","2,212,711","1,106,532","9,508",0,0,5,745,760
93,20250318,DB2R020100,Postpaid Revenue B2B,20250316,2025-03-18,"2,779,864","5,559,728","5,559,658","5,559,658","2,779,864","5,060",0,0,5,718,733
94,20250318,DB2R020100,Postpaid Revenue B2B,20250317,2025-03-18,"2,136,510","4,273,020","4,273,123","4,273,123","2,136,510","12,202",0,0,5,806,821


In [10]:
# txn_metric_list = [
#     'DB1R000100' #Prepay Revenue
#     , 'DB2R000100' #Postpaid Revenue
#     , 'DB2R010100' #Postpaid Revenue B2C
#     , 'DB2R020100' #Postpaid Revenue B2B
#     , 'DB1S000500' #Prepay Usage Subs
#     , 'DB1S000600' #Prepay Revenue Subs
#     , 'DB2S000500' #Postpaid Active Subs
#     , 'DB2S010500' #Postpaid Active Subs B2C
#     , 'DB2S020400' #Postpaid Active Sub B2B
#     ]